# [M_03] PROTOCOL: AUTOMATIC VIDEO FACTORY (FFMPEG)

**PROJECT:** OMNI-OPERATOR-V1  
**ENGINE:** GEMINI 3 FLASH  
**STATUS:** MEDIA_EXTRACTION  

This module is responsible for physical video material processing. We use the `MoviePy` library (powered by `FFmpeg`) to automatically cut and render fragments for Shorts based on structured report from Gemini 3 Flash.

**Operational objectives:**
1. Converting `MM:SS` timestamps to seconds (float).
2. Automatic clip extraction from source file `test_video.mp4`.
3. Applying rendering parameters optimized for mobile devices (H.264).

In [1]:
import os
import sys
from moviepy.video.io.VideoFileClip import VideoFileClip

# 1. WORKING DIRECTORY CORRECTION
if os.getcwd().endswith("notebooks"):
    os.chdir("..")

# Adding src to path to see potential helper modules
sys.path.append(os.path.join(os.getcwd(), "src"))

print(f"LOG: Editing system initialized. ROOT directory: {os.getcwd()}")

LOG: Editing system initialized. ROOT directory: c:\Users\takze\OneDrive\Pulpit\project\omni-operator-v1


## 1. Helper Tools: Time Conversion

The report from Gemini (M_01) provides time in text format `MM:SS`. FFmpeg engine requires precise number of seconds. We implement safe converter.

In [2]:
def timestamp_to_seconds(ts: str) -> float:
    """Converts MM:SS format to seconds (e.g. '01:15' -> 75.0)."""
    try:
        parts = ts.split(':')
        if len(parts) == 2:
            minutes, seconds = map(int, parts)
            return float(minutes * 60 + seconds)
        return 0.0
    except (ValueError, AttributeError):
        print(f"ERR: Invalid timestamp format: {ts}")
        return 0.0

# OPERATIONAL TEST
test_ts = "00:45"
print(f"TEST: {test_ts} -> {timestamp_to_seconds(test_ts)}s")

TEST: 00:45 -> 45.0s


## 2. Production Engine Implementation

We create `produce_shorts` function that takes raw video and list of clips, then performs rendering. Each clip will be saved in dedicated `output/` folder.

In [3]:
def produce_shorts(source_file: str, clips_to_cut: list):
    """
    Physically cuts video fragments based on list of dictionaries.
    """
    output_dir = "output"
    os.makedirs(output_dir, exist_ok=True)
    
    if not os.path.exists(source_file):
        raise FileNotFoundError(f"Source file not found: {source_file}")

    print(f"LOG: Opening source material: {source_file}")
    
    # In MoviePy 2.x we use context manager (with)
    with VideoFileClip(source_file) as video:
        for i, clip_info in enumerate(clips_to_cut, 1):
            start_s = timestamp_to_seconds(clip_info['start'])
            end_s = timestamp_to_seconds(clip_info['end'])
            
            output_path = os.path.join(output_dir, f"short_{i}.mp4")
            
            print(f"LOG: Rendering clip {i} [{clip_info['start']} - {clip_info['end']}]...")

            # Safety Guard: Max 90 seconds per clip
            if (end_s - start_s) > 90:
                print(f"⚠️ WARNING: Clip {i} is too long ({end_s - start_s}s). Trimming to 60s.")
                end_s = start_s + 60

            # NEW METHOD IN v2.x: subclipped() instead of subclip()
            new_clip = video.subclipped(start_s, end_s)
            
            # File save
            new_clip.write_videofile(
                output_path, 
                codec="libx264", 
                audio_codec="aac",
                logger=None 
            )
            print(f"✅ READY: {output_path}")

    print(f"\nLOG: Production completed successfully.")

## 3. Factory Launch (End-to-End Test)

We use data obtained from Gemini 3 Flash in Module 01 to generate first files.

In [4]:
# Simulation of data from analytical report (M_01)
# If you already have analysis_result.clips, you can use that data directly
mock_production_data = [
    {"start": "00:00", "end": "00:20", "hook": "AI visual effect"}, # 20 seconds
    {"start": "00:25", "end": "00:55", "hook": "Framework presentation"} # 30 seconds
]

INPUT_VIDEO = "test_video.mp4"

try:
    produce_shorts(INPUT_VIDEO, mock_production_data)
except Exception as e:
    print(f"❌ PRODUCTION ERROR: {str(e)}")

LOG: Opening source material: test_video.mp4
LOG: Rendering clip 1 [00:00 - 00:20]...
✅ READY: output\short_1.mp4
LOG: Rendering clip 2 [00:25 - 00:55]...
✅ READY: output\short_2.mp4

LOG: Production completed successfully.


## STATUS: MODULE 03 COMPLETED 70%

We now have a working video factory. The system can autonomously manage files on disk.

**Achievements:**
1. FFmpeg integration with logical agent workflow.
2. Automatic generation of MP4 files optimized for social media.
3. Full sovereignty over editing process.

**Task:**
Open `output/` folder in your project and check if `short_1.mp4` and `short_2.mp4` files play correctly.

## 4. Mobile Optimization: 9:16 Framing and Branding

Most source recordings are in 16:9 format. To dominate TikTok and Reels, we must automatically:
1. Rescale video to fill vertical screen.
2. Crop image to center (Center Crop).
3. Apply OPERATORS' FORGE watermark.

In [5]:
from moviepy.video.VideoClip import TextClip, ColorClip
from moviepy.video.compositing.CompositeVideoClip import CompositeVideoClip

def format_to_shorts(input_path: str, output_path: str):
    """Processes horizontal video to vertical 9:16 format with branding."""
    print(f"LOG: Formatting {input_path} to 9:16...")
    
    with VideoFileClip(input_path) as clip:
        # 1. We calculate proportions (target 1080x1920 for 9:16)
        target_ratio = 9/16
        w, h = clip.size
        
        # We scale video so height fits vertical, and width is excessive
        # Then we do Center Crop
        new_h = h
        new_w = int(h * target_ratio)
        
        # MoviePy v2.x: cropped() instead of crop()
        final_clip = clip.cropped(
            x_center=w/2, 
            y_center=h/2, 
            width=new_w, 
            height=new_h
        )

        # 2. ADDING BRANDING (Simple status bar at bottom)
        # We create simple text overlay
        # Note: Requires ImageMagick installed on system for TextClip
        try:
            brand_overlay = ColorClip(
                size=(new_w, 60), 
                color=(139, 0, 0) # Our Dark Red #8B0000
            ).with_duration(final_clip.duration).with_opacity(0.8).with_position(("center", "bottom"))
            
            result = CompositeVideoClip([final_clip, brand_overlay])
        except:
            print("LOG: Skipping branding (no ImageMagick). Rendering clean vertical.")
            result = final_clip

        # 3. FINAL RENDER
        result.write_videofile(
            output_path,
            codec="libx264",
            audio_codec="aac",
            logger=None
        )
        print(f"✅ SHORTS PRODUCED: {output_path}")

# TEST: We process first cut clip
if os.path.exists("output/short_1.mp4"):
    format_to_shorts("output/short_1.mp4", "output/short_1_branded.mp4")

LOG: Formatting output/short_1.mp4 to 9:16...
✅ SHORTS PRODUCED: output/short_1_branded.mp4


## STATUS: STAGE 3 COMPLETED (FACTORY_ACTIVE)

The system has gone through the full path from raw file `test_video.mp4` to vertical, branded Short `short_1_branded.mp4`.

**Stage 3 Achievements:**
- [x] Automatic extraction based on Gemini metadata.
- [x] Conversion to vertical format (Vertical Reframing).
- [x] Branding layer implementation.

**Next stage (M_04): Long-term Memory (Qdrant).**
We will save our analyses and cut clips to vector database, so the system learns your style and knows what it published in the past.